In [1]:
import modin.pandas as pd

In [2]:
df_test = pd.read_csv('http.csv')

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})

2023-02-22 19:14:07,076	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(raylet) /home/vzb403/.local/lib/python3.10/site-packages/ray/dashboard/agent.py:51: DeprecationWarning: There is no current event loop
(raylet)   aiogrpc.init_grpc_aio()
(raylet) Error processing line 1 of /home/vzb403/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "/usr/lib/python3.10/site.py", line 192, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "/home/vzb403/.local/lib/python3.10/site-packages/pandas/__init__.py", line 16, in <module>
(raylet)       raise ImportError(
(raylet)   ImportError: Unable to import required dependencies:
(ray

In [3]:
len(df_test['url'])

23555143

In [4]:
df_test['url'].nunique()

6697

In [32]:
df_test['domain'] = df_test['url'].apply(lambda x: x.split('/')[2])

In [6]:
len(df_test['domain'].unique())

454

In [7]:
key = '39506a81-3038-4932-b461-009df8f3b7d3'
url = 'https://investigate.api.umbrella.com'
import requests
import time

In [8]:
def get_cat(key, url, domain):
    header = {'Authorization':'Bearer '+key,
        'Content-Type':'application/json'}
    
    response = requests.get(url+'/domains/categorization/'+domain+'?showLabels', headers=header).json()

    time.sleep(5)

    return response

In [13]:
get_cat(key, url, 'www.google.comtest')

{'www.google.comtest': {'status': 0,
  'security_categories': [],
  'content_categories': []}}

In [31]:
domainCats = {}

for domain in df_test['domain'].unique():
   try:
      response = get_cat(key, url, domain)
      domainCats[domain] = response[domain]['content_categories']
   except:
      print(response)

In [33]:
import pickle

#with open('domainCats.pkl', 'wb') as handle:
#    pickle.dump(domainCats, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('domainCats.pkl', 'rb') as handle:
    domainCats = pickle.load(handle)


In [34]:
len(domainCats.keys())

454

In [35]:
len(df_test['domain'].unique())

454

In [5]:
df_test['date'] = pd.to_datetime(df_test['date'])

In [11]:
df_test['dayDate'] = df_test['date'].dt.date

In [16]:
df_test['dateString'] = df_test['date'].dt.strftime('%Y%m%d')

In [20]:
df_test['userDate'] = df_test.apply(lambda x: x['user'] + '_' + x['dateString'], axis=1)

In [26]:
df_urlByUserDate = df_test.head(51000).groupby('userDate').count()['url'].to_frame().rename(columns={'url':'count'})

In [28]:
df_urlByUserDate.to_pickle('urlByUserDate.pkl')

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [29]:
import pandas

In [34]:
df_test.head()

,id,date,user,pc,url,content,dayDate,dateString,userDate,domain
0,{U0G4-P5TW04EO-9366IUQF},2010-01-02 06:36:20,EBH0519,PC-9573,http://usbank.com/1930_FIFA_World_Cup/patenaud...,neros printing pastiche four topiary body pink...,2010-01-02,20100102,EBH0519_20100102,usbank.com
1,{G0R5-U4RX40DR-3710WPUY},2010-01-02 06:39:22,EBH0519,PC-9573,http://forbes.com/Chorioactis/urnula.html,balhae fubing guotu namsaeng geng chuzhou ying...,2010-01-02,20100102,EBH0519_20100102,forbes.com
2,{G0Q6-F5CV96RX-0355APNP},2010-01-02 06:39:53,EBH0519,PC-9573,http://coupons.com/BeauchampSharp_Tragedy/darb...,ziyi hazelnuts 34850 stat_area1 namsaeng kucha...,2010-01-02,20100102,EBH0519_20100102,coupons.com
3,{J8X0-C9ZO08VS-9370QIFO},2010-01-02 06:40:56,EBH0519,PC-9573,http://youtube.com/Amanita_ocreata/velosa.html,flambard anselm anselm precentor thurgot visit...,2010-01-02,20100102,EBH0519_20100102,youtube.com
4,{E0J4-D6WM72LS-9186JPKE},2010-01-02 06:47:23,EBH0519,PC-9573,http://twitter.com/Cannon/m198.html,maternus henrici anselm walkelin maternus thur...,2010-01-02,20100102,EBH0519_20100102,twitter.com


In [39]:
df_test['domainCats'] = df_test.apply(lambda x: domainCats[x['domain']], axis=1)


In [43]:
import itertools
set(itertools.chain.from_iterable(domainCats.values()))

{'Adult',
 'Adult Themes',
 'Advertisements',
 'Adware',
 'Arts',
 'Auctions',
 'Automotive',
 'Blogs',
 'Business Services',
 'Business and Industry',
 'Chat',
 'Classifieds',
 'Cloud and Data Centers',
 'Computer Security',
 'Computers and Internet',
 'DIY Projects',
 'Dating',
 'Digital Postcards',
 'Dining and Drinking',
 'Ecommerce/Shopping',
 'Education',
 'Educational Institutions',
 'Entertainment',
 'File Storage',
 'File Transfer Services',
 'Finance',
 'Financial Institutions',
 'Forums/Message boards',
 'Freeware and Shareware',
 'Games',
 'Government',
 'Government and Law',
 'Health and Fitness',
 'Health and Medicine',
 'Humor',
 'Illegal Downloads',
 'Infrastructure',
 'Instant Messaging',
 'Internet Telephony',
 'Job Search',
 'Jobs/Employment',
 'Lingerie and Swimsuits',
 'Lingerie/Bikini',
 'Lotteries',
 'Military',
 'Movies',
 'Music',
 'News',
 'News/Media',
 'Non-Profits',
 'Nudity',
 'Online Communities',
 'Online Document Sharing and Collaboration',
 'Online Mee

In [44]:
insiderCats = ['File Storage',
 'File Transfer Services',
 'Forums/Message boards',
 'Freeware and Shareware',
 'Illegal Downloads',
 'Instant Messaging',
 'Internet Telephony',
 'Job Search',
 'Jobs/Employment',
 'Lingerie and Swimsuits',
 'Lingerie/Bikini',
 'Nudity',
 'Online Document Sharing and Collaboration',
 'Online Storage and Backup',
 'Photo Sharing',
 'Pornography',
 'Professional Networking',
 'Social Networking',
 'Video Sharing',
 'Web Spam',
 'Web-based Email',
 'Webmail']

In [62]:
insiderCats = ['File Storage',
 'File Transfer Services',
 'Forums/Message boards',
 'Internet Telephony',
 'Job Search',
 'Jobs/Employment',
 'Online Document Sharing and Collaboration',
 'Online Storage and Backup',
 'Professional Networking',
 'Web-based Email',
 'Webmail']

In [63]:
def isInsiderWeb(row):
    if any(item in row['domainCats'] for item in insiderCats):
        return 1
    else:
        return 0

df_test['isInsiderWeb'] = df_test.apply(isInsiderWeb, axis=1)

In [70]:
df_test['domainCatsString'] = df_test.apply(lambda x: str(x['domainCats']), axis=1)

In [73]:
df_insiderWebCount = df_test[df_test['isInsiderWeb'] == 1].groupby('userDate').sum()

(raylet) Error processing line 1 of /home/vzb403/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "/usr/lib/python3.10/site.py", line 192, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "/home/vzb403/.local/lib/python3.10/site-packages/pandas/__init__.py", line 16, in <module>
(raylet)       raise ImportError(
(raylet)   ImportError: Unable to import required dependencies:
(raylet)   pytz: No module named 'pytz'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 1 of /home/vzb403/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "/usr/lib/python3.10/site.py", line 192, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "/home/vzb403/.local/lib/python3.10/site-packages/p

In [74]:
df_insiderWebCount.to_pickle('insiderWebCount.pkl')

In [71]:
df_test[df_test['isInsiderWeb'] == 1]['domainCatsString'].value_counts()

(raylet) Error processing line 1 of /home/vzb403/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "/usr/lib/python3.10/site.py", line 192, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "/home/vzb403/.local/lib/python3.10/site-packages/pandas/__init__.py", line 16, in <module>
(raylet)       raise ImportError(
(raylet)   ImportError: Unable to import required dependencies:
(raylet)   pytz: No module named 'pytz'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Spilled 32934 MiB, 2948 objects, write throughput 828 MiB/s.
(raylet) Error processing line 1 of /home/vzb403/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "/usr/lib/python3.10/site.py", line 192, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>

['News/Media', 'Adult Themes', 'Forums/Message boards', 'Research/Reference', 'Online Communities']                     5894
['Games', 'News/Media', 'Forums/Message boards', 'Entertainment']                                                       5690
['Games', 'Software/Technology', 'Forums/Message boards']                                                               4132
['File Storage', 'Business Services', 'Online Document Sharing and Collaboration']                                      4041
['Chat', 'Instant Messaging', 'Software/Technology', 'Internet Telephony']                                              3568
['File Transfer Services']                                                                                              3316
['Jobs/Employment', 'Job Search']                                                                                       3055
['Blogs', 'Business Services', 'Forums/Message boards', 'Online Communities']                                           2829
